In [1]:
from python.simulation import Simulation
import pandas as pd
import os
import time

In [2]:
help(Simulation)

Help on class Simulation in module python.simulation:

class Simulation(python.wrapper.python_lbm.PythonClient)
 |  Method resolution order:
 |      Simulation
 |      python.wrapper.python_lbm.PythonClient
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, lbmFile)
 |      Load a simulation from lbm file
 |      
 |      Arguments:
 |          lbmFile {str} -- File system path to the lbm file
 |  
 |  get_average_names(self)
 |      List the names of the time averaged measurement areas
 |      
 |      Returns:
 |          list -- List of name strings
 |  
 |  get_averages(self, avg_type)
 |      Get the latest time averaged values measured during the simulation
 |      
 |      Arguments:
 |          avg_type {str} -- temperature, velocity or flow
 |      
 |      Returns:
 |          DataFrame -- A dataframe of measured values for all areas
 |  
 |  get_boundary_condition_names(self)
 |      Get a list of named bo

In [3]:
sim = Simulation(f'{os.getcwd()}/problems/ocp/project.lbm')

In [4]:
#sim.get_boundary_condition_names()

In [5]:
sim.get_boundary_conditions('P02R04C07SRV03')

,type,temperature,velocity,normal,rel_pos,tau1,tau2,lambda
715,VoxelType.INLET_RELATIVE,7.436508,"(-0.05, 0, 0)","(-1, 0, 0)","(25, 0, 0)",533.333313,483.383698,0.0
714,VoxelType.INLET_ZERO_GRADIENT,NaN,"(-0.05, 0, 0)","(1, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0


In [6]:
sim.set_time_averaging_period(1.0)
sim.get_time_step()

0.0018193715950474143

In [7]:
sim.get_boundary_conditions()

,type,temperature,velocity,normal,rel_pos,tau1,tau2,lambda
0,VoxelType.FLUID,NaN,"(nan, nan, nan)","(0, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0
1,VoxelType.EMPTY,NaN,"(nan, nan, nan)","(0, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0
2,VoxelType.WALL,NaN,"(nan, nan, nan)","(1, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0
3,VoxelType.WALL,NaN,"(nan, nan, nan)","(-1, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0
4,VoxelType.WALL,NaN,"(nan, nan, nan)","(0, 1, 0)","(0, 0, 0)",0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...
753,VoxelType.INLET_RELATIVE,8.366072,"(0.0666667, 0, 0)","(1, 0, 0)","(-25, 0, 0)",533.333313,483.383698,0.0
754,VoxelType.INLET_ZERO_GRADIENT,NaN,"(0.0666667, 0, 0)","(-1, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0
755,VoxelType.INLET_RELATIVE,8.366072,"(0.0666667, 0, 0)","(1, 0, 0)","(-25, 0, 0)",533.333313,483.383698,0.0
756,VoxelType.INLET_ZERO_GRADIENT,NaN,"(0.0666667, 0, 0)","(-1, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0


In [8]:
# Server mass [kg]
M = 8
# Overall specific heat [J/(kg*K)]
cp_eff = 400
# Specific heat capacity of air [J/(kg*K)]
# https://www.engineeringtoolbox.com/air-specific-heat-capacity-d_705.html
cp_air = 1000
# Overall effective heat transfer coefficient [J/(s*m²*K)]
# https://www.engineeringtoolbox.com/convective-heat-transfer-d_430.html
h = 40
# Server surface area [m²]
A = 0.15
# Average airflow rate through server [kg/s]
q_dot = 0.005 # [m³/s]
rho_air = 1.324 # [kg/m³]
m_dot = q_dot * rho_air
# Time constants [s]
tau1 = M*cp_eff/(h*A)
tau2 = M*cp_eff/(m_dot*cp_air)
lambda1 = 0

In [9]:
# Chassi fan specs from PFR0812DHE fan datasheet
# Max input power in W
Pmax = 25.2
# Max RPM
Nmax = 11000.0
# Operational speed (RPM)
Nop = 8000.0
# Max air flow in CFM
Qmax = 109.7
# Fans per server
Nfans = 2
# Operational power from cube law of fans
Pop = Pmax / (Nmax / Nop)**3 * Nfans
# Assume fan volumetric flow is proportional to power 
# around operating point
Qop = Qmax / (Pmax / Pop)
# Calculate flow in m^3/s per RPM at
Q_per_RPM =  Qop * 0.3048**3 / 60 / Nop

In [10]:
# Kinematic viscosity of air (m^2/s)
nu = 1.568e-5
# Thermal conductivity (kW/m K)
k = 2.624e-5
# Prandtl number of air
Pr = 0.707
# Calculate the expected temperature jump across the servers (convert to kW)
def deltaT(p, q):
    return (p / 1000 * nu) / (q * k * Pr)

In [11]:
srvs = []
# 3 servers per chassi, 10 chassis per rack, 12 racks
for rack in range(1, 13):
    for chassi in range(1, 11):
        for srv in range(1, 4):
            pad = lambda i : str(i).zfill(2)
            srvs += [f'P02R{pad(rack)}C{pad(chassi)}SRV{pad(srv)}']

In [12]:
# Temperatures and volumetric flows should be set from some thermal model
vol_flows = [2000 * Q_per_RPM for _ in range(0, len(srvs))]
p = 45
temps = [deltaT(p, q) for q in vol_flows]
sim.set_boundary_conditions(srvs, temps, vol_flows)

In [13]:
sim.get_boundary_conditions()

,type,temperature,velocity,normal,rel_pos,tau1,tau2,lambda
0,VoxelType.FLUID,NaN,"(nan, nan, nan)","(0, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0
1,VoxelType.EMPTY,NaN,"(nan, nan, nan)","(0, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0
2,VoxelType.WALL,NaN,"(nan, nan, nan)","(1, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0
3,VoxelType.WALL,NaN,"(nan, nan, nan)","(-1, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0
4,VoxelType.WALL,NaN,"(nan, nan, nan)","(0, 1, 0)","(0, 0, 0)",0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...
753,VoxelType.INLET_RELATIVE,3.819558,"(0.0250691, 0, 0)","(1, 0, 0)","(-25, 0, 0)",533.333313,483.383698,0.0
754,VoxelType.INLET_ZERO_GRADIENT,3.819558,"(0.0313363, -0, -0)","(-1, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0
755,VoxelType.INLET_RELATIVE,3.819558,"(0.0313363, 0, 0)","(1, 0, 0)","(-25, 0, 0)",533.333313,483.383698,0.0
756,VoxelType.INLET_ZERO_GRADIENT,3.819558,"(0.0313363, -0, -0)","(-1, 0, 0)","(0, 0, 0)",0.000000,0.000000,0.0


In [14]:
sim.get_time()

datetime.datetime(1970, 1, 1, 1, 0)

In [15]:
for _ in range(0, 60):
    sim.run(1.0)
    start = time.time()
    t = sim.get_averages("temperature")[['time', 'P02R10C05SRV01_inlet']]
    print(t)
    end = time.time()
    print(f'exec time={end - start}')

                        time  P02R10C05SRV01_inlet
0 1970-01-01 01:00:00.001819                   0.0
exec time=0.05423593521118164
                        time  P02R10C05SRV01_inlet
0 1970-01-01 01:00:01.002473             19.474432
exec time=0.05013680458068848
                        time  P02R10C05SRV01_inlet
0 1970-01-01 01:00:02.003127             19.607424
exec time=0.05367588996887207
                        time  P02R10C05SRV01_inlet
0 1970-01-01 01:00:03.003781             19.575811
exec time=0.053336381912231445
                        time  P02R10C05SRV01_inlet
0 1970-01-01 01:00:04.004435             19.496521
exec time=0.05652451515197754
                        time  P02R10C05SRV01_inlet
0 1970-01-01 01:00:05.005089             18.795433
exec time=0.05204510688781738
                        time  P02R10C05SRV01_inlet
0 1970-01-01 01:00:06.005743             18.750095
exec time=0.04990243911743164
                        time  P02R10C05SRV01_inlet
0 1970-01-01 01:00:07.00

In [16]:
sim.get_time()

datetime.datetime(1970, 1, 1, 1, 1, 0, 39243)

In [17]:
sim.get_historical_averages("temperature")

,time,P02HDZ01_out,P02HDZ01_in,P02HDZ03_out,P02HDZ03_in,P02HDZ04_out,P02HDZ04_in,P02HDZ02_out,P02HDZ02_in,P02R11C10SRV03_inlet,...,P02R07C03SRV02_inlet,P02R07C03SRV02_outlet,P02R07C03SRV01_inlet,P02R07C03SRV01_outlet,P02R10C05SRV03_inlet,P02R10C05SRV03_outlet,P02R10C05SRV02_inlet,P02R10C05SRV02_outlet,P02R10C05SRV01_inlet,P02R10C05SRV01_outlet
0,1970-01-01 01:00:59.040408,18.567457,22.54184,18.52174,22.644678,18.538132,22.666227,18.541271,22.498158,22.312382,...,18.490301,23.139515,18.272984,23.315964,19.051582,23.163134,20.095577,23.152317,20.06863,23.127907


In [18]:
# Temperatures and volumetric flows should be set from some thermal model
vol_flows = [2000 * Q_per_RPM for _ in range(0, len(srvs))]
temps = [deltaT(1000, q) for q in vol_flows]
sim.set_boundary_conditions(srvs, temps, vol_flows)

In [19]:
sim.run(60.0)

In [20]:
sim.get_historical_averages("temperature")[['time', 'sensors_racks_10_to_12_out_m']]

KeyError: "['sensors_racks_10_to_12_out_m'] not in index"